In [1]:
import http.client
import json
import pandas as pd
import requests
import config

In [8]:
url = f'https://api.sportmonks.com/v3/football/fixtures?api_token={config.api_key}&include=participants'

response = requests.get(url)
data = response.json()

df = pd.DataFrame(data['data'])

In [9]:
df.head()

,id,sport_id,league_id,season_id,stage_id,group_id,aggregate_id,round_id,state_id,venue_id,name,starting_at,result_info,leg,details,length,placeholder,has_odds,starting_at_timestamp,participants
0,216268,1,271,1273,1086,None,None,23332,5,618,Esbjerg vs OB,2006-03-25 16:00:00,Game ended in draw.,1/1,None,90,False,False,1143302400,"[{'id': 1371, 'sport_id': 1, 'country_id': 320..."
1,216269,1,271,1273,1086,None,None,23329,5,85765,AaB vs København,2005-11-27 16:30:00,Game ended in draw.,1/1,None,90,False,False,1133109000,"[{'id': 1020, 'sport_id': 1, 'country_id': 320..."
2,216270,1,271,1273,1086,None,None,23333,5,85371,Silkeborg vs AGF,2006-03-29 17:00:00,Game ended in draw.,1/1,None,90,False,False,1143651600,"[{'id': 86, 'sport_id': 1, 'country_id': 320, ..."
3,216271,1,271,1273,1086,None,None,23325,5,1436,Nordsjælland vs SønderjyskE,2005-10-23 13:00:00,Game ended in draw.,1/1,None,90,False,False,1130072400,"[{'id': 2394, 'sport_id': 1, 'country_id': 320..."
4,216272,1,271,1273,1086,None,None,23332,5,5661,Horsens vs Viborg,2006-03-19 14:00:00,Game ended in draw.,1/1,None,90,False,False,1142776800,"[{'id': 211, 'sport_id': 1, 'country_id': 320,..."


In [12]:
participants = pd.DataFrame(df['participants'].values.tolist())
participants.head()

,0,1
0,"{'id': 1371, 'sport_id': 1, 'country_id': 320,...","{'id': 1789, 'sport_id': 1, 'country_id': 320,..."
1,"{'id': 1020, 'sport_id': 1, 'country_id': 320,...","{'id': 85, 'sport_id': 1, 'country_id': 320, '..."
2,"{'id': 86, 'sport_id': 1, 'country_id': 320, '...","{'id': 2905, 'sport_id': 1, 'country_id': 320,..."
3,"{'id': 2394, 'sport_id': 1, 'country_id': 320,...","{'id': 390, 'sport_id': 1, 'country_id': 320, ..."
4,"{'id': 211, 'sport_id': 1, 'country_id': 320, ...","{'id': 2447, 'sport_id': 1, 'country_id': 320,..."


In [18]:
home = pd.DataFrame(participants[0].values.tolist())
away = pd.DataFrame(participants[1].values.tolist())

In [19]:
home.head()

,id,sport_id,country_id,venue_id,gender,name,short_code,image_path,founded,type,placeholder,last_played_at,meta
0,1371,1,320,618,male,Esbjerg,ESB,https://cdn.sportmonks.com/images/soccer/teams...,1924,domestic,False,2023-05-07 11:00:00,"{'location': 'home', 'winner': False, 'positio..."
1,1020,1,320,85765,male,AaB,AAB,https://cdn.sportmonks.com/images/soccer/teams...,1885,domestic,False,2023-05-07 10:00:00,"{'location': 'home', 'winner': False, 'positio..."
2,86,1,320,85371,male,Silkeborg,SIL,https://cdn.sportmonks.com/images/soccer/teams...,1917,domestic,False,2023-05-07 12:00:00,"{'location': 'home', 'winner': False, 'positio..."
3,2394,1,320,1436,male,Nordsjælland,FCN,https://cdn.sportmonks.com/images/soccer/teams...,2003,domestic,False,2023-05-08 17:00:00,"{'location': 'home', 'winner': False, 'positio..."
4,211,1,320,5661,male,Horsens,ACH,https://cdn.sportmonks.com/images/soccer/teams...,1994,domestic,False,2023-05-07 10:00:00,"{'location': 'home', 'winner': False, 'positio..."


In [20]:
away.head()

,id,sport_id,country_id,venue_id,gender,name,short_code,image_path,founded,type,placeholder,last_played_at,meta
0,1789,1,320,214349,male,OB,ODE,https://cdn.sportmonks.com/images/soccer/teams...,1889,domestic,False,2023-05-07 12:00:00,"{'location': 'away', 'winner': False, 'positio..."
1,85,1,320,5655,male,København,COP,https://cdn.sportmonks.com/images/soccer/teams...,1992,domestic,False,2023-05-08 17:00:00,"{'location': 'away', 'winner': False, 'positio..."
2,2905,1,320,1708,male,AGF,AGF,https://cdn.sportmonks.com/images/soccer/teams...,1902,domestic,False,2023-05-07 16:00:00,"{'location': 'away', 'winner': False, 'positio..."
3,390,1,320,5653,male,SønderjyskE,SON,https://cdn.sportmonks.com/images/soccer/teams...,2004,domestic,False,2023-05-06 11:00:00,"{'location': 'away', 'winner': False, 'positio..."
4,2447,1,320,1467,male,Viborg,None,https://cdn.sportmonks.com/images/soccer/teams...,1896,domestic,False,2023-05-07 16:00:00,"{'location': 'away', 'winner': False, 'positio..."
